In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(20, '/home/nicolas/anaconda2/lib/python2.7/site-packages')
import datashader as ds
import datashader.transfer_functions as tf

data = pd.read_csv("./asm_data_for_ml.txt", sep='\t')
data_plot = pd.DataFrame({'x':data['rateC'] / data['rateA'], 'y':data['rate']})
data['rateCA'] = data.rateC / data.rateA
data['rateCB'] = data.rateC / data.rateB
data['rateBA'] = data.rateB / data.rateA

In [ ]:
from pomegranate import GeneralMixtureModel
from pomegranate import NormalDistribution
%pylab inline


X = np.c_[data.orbit, data.rateA, data.rateB, data.rateC, data.rate, data.rateCA]

gmm = GeneralMixtureModel(NormalDistribution, n_components=3)
#gmm = GeneralMixtureModel([NormalDistribution(5, 2), NormalDistribution(1, 2)])
gmm.fit(X)
preds = gmm.predict(X)
probs = gmm.predict_proba(X)
print 'probs=', probs

data['preds'] = pd.Series(preds).astype("category")
color_key = ['red', 'blue', 'yellow']

cvs = ds.Canvas(plot_height=300, plot_width=900, x_range=[0, 5], y_range=[0, 200])

agg = cvs.points(data, 'rateCA', 'rate', ds.count_cat('preds'))

tf.colorize(agg, color_key)


In [ ]:
# Hmm with 3 components:

# looks better with data.rateCA in X
# looks like 3 is better than 4 components

from pomegranate import HiddenMarkovModel

X = np.c_[data.orbit, data.rateA, data.rateB, data.rateC, data.rate, data.rateCA]

clf = HiddenMarkovModel()

In [ ]:
clf.fit(X)

In [ ]:
XXX

preds = clf.predict(X)
probs = clf.predict_proba(X)

print clf.score(X)

color_key = ['red', 'green', 'blue', 'yellow']

data['preds'] = pd.Series(preds).astype("category")

cvs = ds.Canvas(plot_height=300, plot_width=900, x_range=[0, 5], y_range=[0, 200])
agg = cvs.points(data, 'rateCA', 'rate', ds.count_cat('preds'))

tf.colorize(agg, color_key)